### Ejercicio con el modelo Naive Bayes
El objetivo es crear un clasificador de reseñas de la tienda de Google Play.

### Importo librerías y base de datos

In [52]:
import pandas as pd
import numpy as np
import random

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_iris
from sklearn.datasets import fetch_20newsgroups

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report, confusion_matrix

import joblib
import warnings
warnings.filterwarnings('ignore')

In [53]:
archivo = "https://raw.githubusercontent.com/4GeeksAcademy/naive-bayes-project-tutorial/main/playstore_reviews.csv"
df = pd.read_csv(archivo, sep=",")

df.head()

,package_name,review,polarity
0,com.facebook.katana,privacy at least put some option appear offli...,0
1,com.facebook.katana,"messenger issues ever since the last update, ...",0
2,com.facebook.katana,profile any time my wife or anybody has more ...,0
3,com.facebook.katana,the new features suck for those of us who don...,0
4,com.facebook.katana,forced reload on uploading pic on replying co...,0


In [54]:
df.info

<bound method DataFrame.info of              package_name                                             review  \
0     com.facebook.katana   privacy at least put some option appear offli...   
1     com.facebook.katana   messenger issues ever since the last update, ...   
2     com.facebook.katana   profile any time my wife or anybody has more ...   
3     com.facebook.katana   the new features suck for those of us who don...   
4     com.facebook.katana   forced reload on uploading pic on replying co...   
..                    ...                                                ...   
886  com.rovio.angrybirds   loved it i loooooooooooooovvved it because it...   
887  com.rovio.angrybirds   all time legendary game the birthday party le...   
888  com.rovio.angrybirds   ads are way to heavy listen to the bad review...   
889  com.rovio.angrybirds   fun works perfectly well. ads aren't as annoy...   
890  com.rovio.angrybirds   they're everywhere i see angry birds everywhe...   

     po

In [55]:
df.shape

(891, 3)

### Tipología de variables

In [56]:
target_col = "polarity"

num_cols = [c for c in df.columns 
            if pd.api.types.is_numeric_dtype(df[c])]

cat_cols = [c for c in df.columns 
            if pd.api.types.is_string_dtype(df[c]) 
            and c not in [target_col]]

num_cols[:10], cat_cols[:10]

print(f"La categoría numérica es: {num_cols}")
print(f"La categoría categórica es: {cat_cols}")

La categoría numérica es: ['polarity']
La categoría categórica es: ['package_name', 'review']


### Limpieza de datos

In [57]:
duplicados = df[df.duplicated(keep=False)]
total_duplicados = df.duplicated().sum()
print(f"Número de filas duplicadas: {total_duplicados}")

Número de filas duplicadas: 0


In [58]:
df = df.drop(columns=["package_name"])

No existen duplicados y no hay variables numéricas continuas, con lo cuál no merece la pena analizar outliers en este dataset. La columna "polarity" es de tipo binario, por tanto no puede tener outliers.

La columna "package_name" ha sido eliminada, ya que no aporta información a este modelo.  Me centro en la columna "review" para ver si existen textos vacíos. 

In [59]:
df['review_length'] = df['review'].str.len()
df['review_length'].describe()

count    891.000000
mean     231.873176
std      129.620763
min        6.000000
25%      145.000000
50%      210.000000
75%      292.500000
max      910.000000
Name: review_length, dtype: float64

In [60]:
# Compruebo las reviews demasiado cortas para ver si pueden afectar el dataset
df[df['review_length'] < 20].head()

,review,polarity,review_length
557,awesome,1,11
761,aa nice,1,8
766,loved it,1,10
767,good,1,6
770,ads problem,0,13


In [61]:
# Elimino espacios y convierto a minúsculas el texto

df["review"] = df["review"].str.strip().str.lower()

Las reviews cortas entran dentro lo esperado en este tipo de reviews y son razonables en el contexto de reseñas de Google Play, con lo cual, las mantengo. Únicamente limpio el formato del texto para mayor legibilidad.

### Hacemos el entrenamiento

In [62]:

X = df['review'] 
y = df['polarity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [63]:
X_train.head()

331    just did the latest update on viber and yet ag...
733    keeps crashing it only works well in extreme d...
382    the fail boat has arrived the 6.0 version is t...
704    superfast, just as i remember it ! opera mini ...
813    installed and immediately deleted this crap i ...
Name: review, dtype: object

In [64]:
# Utilizo TF_IDF en lugar de CountVectorizer ya que existen reviews largas. Añado unigrams y bigrams para hacer más rico el accuracy
vectorizer = TfidfVectorizer(lowercase=True,stop_words='english', max_df=0.95,min_df=2, ngram_range=(1,2))

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


Voy a utilizar el modelo MultinomialNB ya que se usa para conteos y modelos de clasificación de textos y funciona bien con TF_IDF. Descarto:

-BernouilliNB: bueno para datos binarios (0/1), pero no para recuentos de palabras. "Polarity" pese a ser una columna binaria, es el target, no el objeto de estudio.

-GaussianNB: recomendable para números continuos, ausentes en este dataframe.

In [65]:
# Uso MultinomialNB al estar diseñado para texto.
model = MultinomialNB()
model.fit(X_train_vec, y_train)


,alpha,1.0
,force_alpha,True
,fit_prior,True
,class_prior,None


In [66]:
y_pred = model.predict(X_test_vec)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nMatriz de Confusión:\n", confusion_matrix(y_test, y_pred))



Accuracy: 0.8156424581005587

Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.95      0.88       126
           1       0.81      0.49      0.61        53

    accuracy                           0.82       179
   macro avg       0.81      0.72      0.75       179
weighted avg       0.82      0.82      0.80       179


Matriz de Confusión:
 [[120   6]
 [ 27  26]]


El modelo interpreta 120 reseñas negativas bien clasificadas y 6 negativas que el modelo cree que son positivos. El rendimiento sería aceptable para detectar quejas.
Hay 26 positivos correctamente clasificados y 27 falsos negativos (positivos que el modelo interpreta como negativos). El f1-score es de 0.61, con lo cuál es aún débil para detectar las reseñas positivas.

Aunque el ejercicio pide optimizar el modelo con random forest, no lo hago puesto que no existen variables numéricas estructuradas en columnas, sólo texto.

In [67]:
# Predicción con el modelo

texto_pos = ["This app is awesome, I love it!"]

X_vec_pos = vectorizer.transform(texto_pos)
pred_pos = model.predict(X_vec_pos)

print("Predicción (positiva):", pred_pos)

texto_neg = ["This app keeps crashing and it's completely useless."]

X_vec_neg = vectorizer.transform(texto_neg)
pred_neg = model.predict(X_vec_neg)

print("Predicción (negativa):", pred_neg)

Predicción (positiva): [1]
Predicción (negativa): [0]


### Guardo el modelo

In [68]:
joblib.dump(model, "modelo_naive_bayes.pkl")
joblib.dump(vectorizer, "vectorizer_tfidf.pkl")


['vectorizer_tfidf.pkl']